In [1]:
from methods import *
import re
import benepar
import pandas as pd
import nltk
import nltk.draw
from nltk.tree import Tree
from nltk.draw.tree import TreeView
import os
from IPython.display import display
# import spacy
from  icecream import ic
from scipy.stats import chisquare

In [2]:
parser = benepar.Parser("benepar_en3")
ans = valid_sentences('COCA/text_academic_rpe/w_acad_1990.txt', english_processor, english_indicator)
df = pd.DataFrame({"sentence":ans})
tree_gen = parser.parse_sents([benepar.InputSentence(s) for s in ans])
trees = []
for t in tree_gen:
  trees.append(t)

/Users/oliverweizel/.pyenv/versions/3.12.7/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/oliverweizel/.pyenv/versions/3.12.7/lib/python3.12/site-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

NEW8


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/Users/oliverweizel/.pyenv/versions/3.12.7/lib/python3.12/site-packages/torch/distributions/distribution.py:56: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


In [5]:
parent_tree = []
parent_text = []
parent_type = []
left_tree = []
left_text = []
left_type = []
cc_tree = []
cc_text = []
right_tree = []
right_text = []
right_type = []


for t in trees:
  ic('full: '+ pp(t))
  for tp in t.treepositions():
    if isinstance(t[tp], str):
      continue
    if t[tp].label() == 'CC':
      parent = t[tp[:-1]]
      relative_tp = tp[-1]
      # ic(relative_tp)
      # ic(parent.treepositions())
      try:
        _ = right = parent[relative_tp + 1]
      except:
        print("bad conjunction")
        ic(pp(parent))
        continue
      ic('parent: ' + pp(parent))
      left = parent[0]
      ic('left: ' + pp(left))
      conj = parent[relative_tp]
      ic('conj: ' + pp(conj))
      ic('conjunction: ' + ' '.join(conj.leaves()))
      right = parent[relative_tp + 1]
      ic('right: ' + pp(right))
      parent_tree.append(parent)
      parent_text.append(text(parent))
      parent_type.append(parent.label())
      left_tree.append(left)
      left_text.append(text(left))
      left_type.append(left.label())
      right_tree.append(right)
      right_text.append(text(right))
      right_type.append(right.label())
      cc_tree.append(conj)
      cc_text.append(text(conj))


IndentationError: unexpected indent (2212511085.py, line 2)

In [ ]:
df = pd.DataFrame({'parent_tree':parent_tree, 'parent_text':parent_text, 'parent_type':parent_type, 'left_tree':left_tree, 'left_text':left_text, 'left_type':left_type, 'right_tree':right_tree, 'right_text':right_text, 'right_type':right_type, 'cc_tree':cc_tree, 'cc_text':cc_text})
df['same_type'] = df.apply(lambda row: row['left_type'] == row['right_type'], axis=1)
print(df['cc_text'].value_counts())
res = chisquare(df['cc_text'].value_counts())
print(res)
print(df.head())